## Create Sweep

In [1]:
import wandb

device = 'cuda'
sweep_config = {
    "method": "grid",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        # awkward repeat
        "seed": {"values":[1,2,3]},     
        # Dataset:
        "nb_input": {"value": 20},
        "nb_output": {"value": 10},
        "nb_steps": {"value": 50},
        "nb_data_samples": {"value": 1000},
        "dim_manifold": {"value": 2},
        "alpha": {"values": [0.5, 1.0, 1.5, 2, 2.5]},  # Randman alpha parameter
        # SNN:  
        "nb_hidden": {"value": 10},
        "learn_beta": {"value": False},        
        # Training:
        "std": {"value": 0.05},
        "epochs": {"value":50},             
        "batch_size": {"value": 256},
        # Optimization:
        "loss_fn": {"value": "cross-entropy"},
        "optimizer": {"value": "Adam"},
        "lr": {"value": 0.01},
        "regularization": {"value": "none"},
        # Evolution Strategy:
        "nb_model_samples": {"value": 20}, 
        "mirror": {"value": True},
    },
}

sweep_id = wandb.sweep(sweep_config, entity="DarwinNeuron", project="Test")

Create sweep with ID: iriica7w
Sweep URL: https://wandb.ai/DarwinNeuron/Test/sweeps/iriica7w


## 🧹Sweep 

In [ ]:
import torch, wandb, os
import torch.optim as optim
from torch.nn.functional import cross_entropy

from src.RandmanFunctions import read_randman_dataset, split_and_load
from src.Models import RandmanSNN
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.Training import train_loop_snn
from src.Utilities import init_result_csv, set_seed
from src.LandscapeAnalysis import LossSurfacePlotter

device = 'cuda'

def run_config(config=None):
    with torch.no_grad(), wandb.init(
        config=config, 
    ) as run:        
        config = wandb.config
        
        # update current run_name
        keys = ["seed", "std", "batch_size", "lr", "nb_model_samples"]
        sorted_items = [f"{k}{getattr(config, k)}" for k in sorted(keys)]
        run.name = "-".join(sorted_items)
        
        # setting up local csv recording (optional)
        config_dict = dict(run.config)
        config_dict['run_id'] = run.id
        result_path, _, _ = init_result_csv(config_dict, run.project)
        
        # initialize Evolution Strategy instance
        set_seed(config.seed)
        es_model = ESModel(
            RandmanSNN,
            config.nb_input,
            config.nb_hidden,
            config.nb_output,
            0.95,
            sample_size=config.nb_model_samples,
            param_std=config.std,
            Optimizer=optim.Adam,
            lr=config.lr,
            device=device,
            mirror=config.mirror,
        )

        # load dataset        
        train_loader, val_loader = split_and_load(read_randman_dataset(
            run.config.nb_output,
            run.config.nb_input,
            run.config.nb_steps,
            run.config.nb_data_samples,
            run.config.dim_manifold,
            run.config.alpha), run.config.batch_size)
        
        # loss surface plotter
        plotter_dir = f"results/{run.project}/runs/{run.id}/"
        os.makedirs(plotter_dir, exist_ok=True)
        loss_plotter = LossSurfacePlotter(plotter_dir+"illuminated_loss_surface.npz")

        # epochs
        for epoch in range(config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(es_model, train_loader, val_loader, cross_entropy, device, run, epoch, result_path, loss_plotter)

# set sweep_id if picking up a sweep:
sweep_id = 'iriica7w'
wandb.agent(sweep_id, run_config, entity="DarwinNeuron", project="Test")

wandb: Agent Starting Run: icuh0v7u with config:
wandb: 	alpha: 0.5
wandb: 	batch_size: 256
wandb: 	dim_manifold: 2
wandb: 	epochs: 50
wandb: 	learn_beta: False
wandb: 	loss_fn: cross-entropy
wandb: 	lr: 0.01
wandb: 	mirror: True
wandb: 	nb_data_samples: 1000
wandb: 	nb_hidden: 10
wandb: 	nb_input: 20
wandb: 	nb_model_samples: 20
wandb: 	nb_output: 10
wandb: 	nb_steps: 50
wandb: 	optimizer: Adam
wandb: 	regularization: none
wandb: 	seed: 2
wandb: 	std: 0.05
wandb: Currently logged in as: yixing (DarwinNeuron) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 0
-------------------------------
batch 0, loss: 2.340807, accuracy: 9.4%
Accuracy: 11.2%, Avg loss: 2.331013 

batch 1, loss: 2.335780, accuracy: 11.3%
Accuracy: 12.5%, Avg loss: 2.326868 

batch 2, loss: 2.330676, accuracy: 12.9%
Accuracy: 12.8%, Avg loss: 2.322236 

batch 3, loss: 2.312560, accuracy: 13.7%
Accuracy: 12.3%, Avg loss: 2.321111 

batch 4, loss: 2.324600, accuracy: 9.4%
Accuracy: 12.4%, Avg loss: 2.318663 

batch 5, loss: 2.311715, accuracy: 12.1%
Accuracy: 12.3%, Avg loss: 2.316075 

batch 6, loss: 2.322734, accuracy: 9.0%
Accuracy: 11.9%, Avg loss: 2.313597 

batch 7, loss: 2.327006, accuracy: 9.0%
Accuracy: 11.6%, Avg loss: 2.312983 

batch 8, loss: 2.302073, accuracy: 13.7%
Accuracy: 11.5%, Avg loss: 2.312065 

batch 9, loss: 2.296862, accuracy: 14.1%
Accuracy: 11.5%, Avg loss: 2.310560 

batch 10, loss: 2.310596, accuracy: 11.3%
Accuracy: 11.5%, Avg loss: 2.309715 

batch 11, loss: 2.310224, accuracy: 8.6%
Accuracy: 11.2%, Avg loss: 2.309450 

batch 12, loss:

## Single Run (for old people)


In [ ]:
import torch, wandb, os
import torch.optim as optim
from torch.nn.functional import cross_entropy

from src.RandmanFunctions import read_randman_dataset, split_and_load
from src.Models import RandmanSNN
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.Training import train_loop_snn
from src.Utilities import init_result_csv, set_seed
from src.LandscapeAnalysis import LossSurfacePlotter

device = 'cuda'
@torch.no_grad()
def train_snn():
    run_name = "plotter-test"
    config = {  # Dataset:
        "nb_input": 10,
        "nb_output": 10,
        "nb_steps": 50,
        "nb_data_samples": 1000,
        "dim_manifold": 2,
        "alpha": 2.0,
        # SNN:
        "nb_hidden": 10,
        "learn_beta": False,
        # Evolution Strategy:
        "nb_model_samples": 20,
        "mirror": True,
        # Training:
        "std": 0.05,
        "epochs": 50,
        "batch_size": 256,
        # Optimization:
        "loss": "cross-entropy",
        "optimizer": "Adam",
        "lr": 0.01,
        "regularization": "none",
    }
    with wandb.init(
        entity="DarwinNeuron", project="Test", name=run_name, config=config
    ) as run:
        # update current run_name
        keys = ["std", "batch_size", "lr", "nb_model_samples"]
        sorted_items = [f"{k}{getattr(run.config, k)}" for k in sorted(keys)]
        run.name = "-".join(sorted_items)
        
        # setting up local csv recording (optional)
        result_path, _, _ = init_result_csv(dict(run.config), run.project)

        # initialize Evolution Strategy instance
        es_model = ESModel(
            RandmanSNN,
            run.config.nb_input,
            run.config.nb_hidden,
            run.config.nb_output,
            0.95,
            sample_size=run.config.nb_model_samples,
            param_std=run.config.std,
            Optimizer=optim.Adam,
            lr=run.config.lr,
            device=device,
            mirror=run.config.mirror,
        )

        # load dataset
        train_loader, val_loader = split_and_load(
            read_randman_dataset(
                run.config.nb_output,
                run.config.nb_input,
                run.config.nb_steps,
                run.config.nb_data_samples,
                run.config.dim_manifold,
                run.config.alpha,
            ),
            run.config.batch_size,
        )
        
        # loss surface plotter
        plotter_dir = f"results/{run.project}/runs/{run.id}/"
        os.makedirs(plotter_dir, exist_ok=True)
        loss_plotter = LossSurfacePlotter(plotter_dir+"illuminated_loss_surface.npz")

        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")

            # train the model
            train_loop_snn(es_model, train_loader, val_loader, cross_entropy, device, run, epoch, loss_plotter=loss_plotter)

train_snn()

Epoch 0
-------------------------------
batch 0, loss: 2.314266, accuracy: 8.2%
Accuracy: 8.7%, Avg loss: 2.308529 

batch 1, loss: 2.307100, accuracy: 8.6%
Accuracy: 8.8%, Avg loss: 2.307101 

batch 2, loss: 2.302372, accuracy: 9.4%
Accuracy: 9.0%, Avg loss: 2.305346 

batch 3, loss: 2.305645, accuracy: 13.7%
Accuracy: 9.4%, Avg loss: 2.303466 

batch 4, loss: 2.305126, accuracy: 8.2%
Accuracy: 9.5%, Avg loss: 2.303110 

batch 5, loss: 2.306287, accuracy: 10.2%
Accuracy: 9.8%, Avg loss: 2.301988 

batch 6, loss: 2.308053, accuracy: 9.0%
Accuracy: 9.9%, Avg loss: 2.302093 

batch 7, loss: 2.306416, accuracy: 9.0%
Accuracy: 10.1%, Avg loss: 2.301707 

batch 8, loss: 2.305209, accuracy: 10.5%
Accuracy: 10.1%, Avg loss: 2.301621 

batch 9, loss: 2.301850, accuracy: 10.9%
Accuracy: 10.1%, Avg loss: 2.301789 

batch 10, loss: 2.299750, accuracy: 8.2%
Accuracy: 10.2%, Avg loss: 2.301653 



Traceback (most recent call last):
  File "C:\Users\w1886\AppData\Local\Temp\ipykernel_29544\3703992940.py", line 88, in train_snn
    train_loop_snn(es_model, train_loader, val_loader, cross_entropy, device, run, epoch, loss_plotter=loss_plotter)
  File "c:\Users\w1886\OneDrive\My Documents\Projects\darwin_neuron\src\Training.py", line 66, in train_loop_snn
    log_model(es_model, run)
  File "c:\Users\w1886\OneDrive\My Documents\Projects\darwin_neuron\src\Training.py", line 15, in log_model
    run.log_model(path=filename, name=model_name)
  File "c:\Users\w1886\anaconda3\envs\darwin\Lib\site-packages\wandb\sdk\wandb_run.py", line 398, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\w1886\anaconda3\envs\darwin\Lib\site-packages\wandb\sdk\wandb_run.py", line 456, in wrapper_fn
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\w1886\anaconda3\envs\darwin\Lib\site-packages\wandb\sdk\wa

## Build a plot-builder

In [ ]:
import numpy as np
from numpy.random import rand
from scipy.interpolate import griddata
from scipy.spatial import cKDTree
from src.LandscapeAnalysis import get_orthogonal_vectors, illuminate_loss_surface_2d
import plotly.graph_objects as go
from src.LandscapeAnalysis import LossSurfacePlotter


(2, 2) (2,) newland.npz 2


In [ ]:
import numpy as np
def ball(x):
    return np.sqrt(np.sum(x**2))
def test():
    center = np.array([0.0, 0.0])
    resolution = 20
    range_lim = 1
    
    lsp = LossSurfacePlotter()
    lsp.illuminate_2d(center, ball,range_lim, resolution)
    
    # add another center
    center2 = np.array([10, 0])
    lsp.illuminate_2d(center2, ball, range_lim, resolution)
    
    fig = lsp.get_plot(resolution=512)
    fig.show()
test()

In [15]:

from numpy.random import rand
import numpy as np

def try_cKDTree():
    points = np.array([[0,0],[0,1],[1,0],[1,1]])
    tree = cKDTree(points)
    
    query = np.array([[0,0],[0.5,1]])
    distances, _ = tree.query(query,k=1)
    print(distances)
try_cKDTree()
    

[0.  0.5]
